# Dijkstra’s algorithm

Given a weighted graph, find the shortest path from the initial node (`1`) to all other nodes.

In [ ]:
import Numeric.LinearAlgebra

vertexMatrix = (12><12)
  [ 0, 3, 5, 3, 0, 1, 2, 1, 0, 0, 3, 5
  , 3, 0, 0, 5, 1, 4, 2, 0, 0, 0, 4, 0
  , 5, 0, 0, 0, 0, 5, 1, 1, 3, 0, 5, 5
  , 3, 5, 0, 0, 5, 0, 1, 0, 0, 2, 0, 0
  , 0, 1, 0, 5, 0, 4, 0, 2, 5, 4, 0, 3
  , 1, 4, 5, 0, 4, 0, 0, 4, 5, 0, 0, 0
  , 2, 2, 1, 1, 0, 0, 0, 4, 0, 2, 0, 0
  , 1, 0, 1, 0, 2, 4, 4, 0, 1, 0, 4, 4
  , 0, 0, 3, 0, 5, 5, 0, 1, 0, 0, 5, 0
  , 0, 0, 0, 2, 4, 0, 2, 0, 0, 0, 0, 3
  , 3, 4, 5, 0, 0, 0, 0, 4, 5, 0, 0, 0
  , 5, 0, 5, 0, 3, 0, 0, 4, 0, 3, 0, 0 ]
  
adjacent :: Int -> Int -> Bool
adjacent current node = vertexMatrix ! (node - 1) ! (current - 1) /= 0.0

distance :: Int -> Int -> Double
distance current node = vertexMatrix ! (current - 1) ! (node - 1)

A node is given a `Permanent` distance value if it is equal to the shortest distance to this node. Otherwise, a node is given a `Temporary` distance value.

In [ ]:
import Data.Map.Strict (Map)
import qualified Data.Map.Strict as Map

data DistanceValue = Temporary Double | Permanent Double
  deriving (Eq)
  
infinity = 1 / 0

instance Show DistanceValue where
  show (Temporary d) = if d == (1 / 0) then "\\infty" else show d
  show (Permanent d) = show d ++ "+"

distnum (Temporary dist) = dist
distnum (Permanent dist) = dist

temporary (Temporary _) = True
temporary _ = False

We'll need to keep track of the shortest distance _calculated so far_. Initially, the shortest path from node `0` is going to be `Infinity` for every node but `0` (since the shortest distance from `0` to `0` is 0).

In [ ]:
-- (Current node, distance values)
type State = (Int, Map Int DistanceValue)

state :: State
state = (1, Map.fromList ((1, Permanent 0.0) : [(i, Temporary infinity) | i <- [2..12]]))

We start every iteration from updating the `Temporary` distance values of nodes adjacent to `current` to `min(distance to node, distance to current + distance from current to node)`

In [ ]:
updateAdjacent :: State -> State
updateAdjacent (current, ds) = (current, Map.mapWithKey updateDist ds)
  where
    updateDist node (Temporary d)
      | adjacent current node =
          Temporary $ min d ((distnum $ ds Map.! current) + (distance current node))
      | otherwise = Temporary d
    updateDist _ permanent = permanent

Then we find the node with the shortest `Temporary` distance and change it to a `Permanent` value. This node is selected as the `current` one for the next iteration.

If there are multiple nodes sharing the smallest distance value, the one with smallest index is chosen.

In [ ]:
promoteMinAdjacent :: State -> State
promoteMinAdjacent (current, ds) = (minNode, promote minNode)
  where
    promote node = Map.adjust (\(Temporary d) -> Permanent d) node ds
    -- Use a left fold to choose a min distance node with the smallest index.
    (minNode, _) = Map.foldlWithKey findMin (0, infinity) ds
    findMin (minNode, minDist) node (Temporary d)
      | d < minDist = (node, d)
      | otherwise = (minNode, minDist)
    findMin acc _ _ = acc

The iteration is terminated when all nodes have `Permanent` distance values.

In [ ]:
shouldTerminate :: State -> Bool
shouldTerminate (current, ds) = Map.null $ Map.filter temporary ds

Finally, we iterate over the vertex matrix, printing state after each iteration. There's an output template for this particular task that I'm going to follow:

In [ ]:
import IHaskell.Display (DisplayData, many, latex)
import Data.List (intercalate, (\\))

explainIteration :: State -> State -> [DisplayData]
explainIteration (prevnode, prevds) (state@(current, ds)) =
  printHead ++ printAdjacent : printNewDists ++ printPromoted : printTail
      where
        printHead
          | prevnode == 1 = [latex "$l(x_1) = 0^+;\\ l(x_i) = \\infty$"]
          | otherwise = []
        printTail
          | shouldTerminate state = [latex "Все пометки постоянные."]
          | otherwise = []
        -- print promoted distance
        printPromoted = latex $ "Постоянную пометку получает $min[l(x_i)] = l(x_{" ++ show permNode ++ "}) = " ++
          show permDist ++ "$, $p = " ++ show permNode ++ "$"
        [(permNode, permDist)] = permanent ds \\ permanent prevds
        permanent = Map.toList . (Map.filter (not . temporary))
        -- print new distances
        printNewDists = pure $ latex $ intercalate ", " $ Map.foldrWithKey newDist [] prevds
        newDist node (Temporary d) acc
          | adjacent node prevnode = (
              "$l(x_{" ++ show node ++ "}) = min[" ++ showDist d ++ ", " ++ show (prevds Map.! prevnode) ++
              show (distance prevnode node) ++ "] = " ++
              show (ds Map.! node) ++ "$"
              ) : acc
          | otherwise = acc
        newDist _ _ acc = acc
        showDist d = if d == (1 / 0) then "\\infty" else show d
        -- print a set of adjacent nodes
        printAdjacent = latex $ "Гр = $\\{" ++ asSet adjacentNodes ++ "\\}$ – " ++ case asSet tempAdjacentNodes of
          [] -> "все вершины имеют постоянные пометки."
          set -> "временные пометки имеют вершины $" ++ set ++ "$, уточним их."
        asSet distmap = intercalate ", " (Map.foldrWithKey
          (\node _ acc -> ("x_{" ++ show node ++ "}") : acc) [] distmap)
        adjacentNodes = Map.filterWithKey (\node _ -> adjacent prevnode node) prevds
        tempAdjacentNodes = Map.filter temporary adjacentNodes

This may not be the prettiest or most efficient code, but it gets the job done. Now, we can conveniently wrap actual iteration and logging into a `Writer` monad: 

In [ ]:
import IHaskell.Display (Display, display)
import Control.Monad.Writer

singleIter :: State -> Writer [DisplayData] State  
singleIter s = do
    let s' = (promoteMinAdjacent . updateAdjacent) s
    tell $ explainIteration s s'
    return s'

logIterations :: Writer [DisplayData] State  
logIterations = go state
  where
    go s
      | shouldTerminate s = return s
      | otherwise = singleIter s >>= go
      
iter :: IO Display
iter = do
  let ((_, s), log) = runWriter logIterations
  display $ log ++ [latex $ intercalate ", " $ Map.foldrWithKey displayNode [] s]
    where
      displayNode node dist acc = ("[1 $\\rightarrow$ " ++ show node ++ "] = " ++ show dist) : acc
      
iter